In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_breusch_godfrey
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import OLSInfluence, variance_inflation_factor
from scipy.stats import shapiro
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df is your dataframe and 'y' is your target variable, 'X' is your feature set
X = df.drop('target', axis=1)
y = df['target']

# Adding a constant for OLS
X_const = sm.add_constant(X)

# Fitting the OLS model
ols_model = sm.OLS(y, X_const).fit()

# Checking homoskedasticity using residuals vs fitted plot
sns.residplot(x=ols_model.fittedvalues, y=ols_model.resid, lowess=True)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted')
plt.show()

# ACF/PACF plots for autocorrelation examination
sm.graphics.tsa.plot_acf(ols_model.resid, lags=40)
sm.graphics.tsa.plot_pacf(ols_model.resid, lags=40)
plt.show()

# Durbin-Watson Test for correlation
dw_test = durbin_watson(ols_model.resid)
print(f'Durbin-Watson Test statistic: {dw_test}')

# Calculating VIF for multicollinearity
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]

print(vif_data)

# Detecting influential points
influence = OLSInfluence(ols_model)
influential_points = influence.summary_frame()
print(influential_points[['cooks_d', 'standard_resid', 'hat_diag']])

# Removing influential points (example threshold: Cook's D > 4/n)
n = len(df)
df_cleaned = df[influential_points['cooks_d'] <= 4/n]

# Re-fitting OLS after removing influential points
X_cleaned = df_cleaned.drop('target', axis=1)
y_cleaned = df_cleaned['target']
X_cleaned_const = sm.add_constant(X_cleaned)

ols_model_cleaned = sm.OLS(y_cleaned, X_cleaned_const).fit()

# Checking normality of residuals
residuals = ols_model_cleaned.resid

# QQ plot
sm.qqplot(residuals, line='45')
plt.title('QQ Plot of Residuals')
plt.show()

# Shapiro-Wilk Test
shapiro_test = shapiro(residuals)
print(f'Shapiro-Wilk Test statistic: {shapiro_test[0]}, p-value: {shapiro_test[1]}')

# Checking homoskedasticity after removing influential points
sns.residplot(x=ols_model_cleaned.fittedvalues, y=ols_model_cleaned.resid, lowess=True)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted (Cleaned)')
plt.show()

# Recalculating ACF/PACF plots, Durbin-Watson Test, and VIF after removing influential points
sm.graphics.tsa.plot_acf(ols_model_cleaned.resid, lags=40)
sm.graphics.tsa.plot_pacf(ols_model_cleaned.resid, lags=40)
plt.show()

dw_test_cleaned = durbin_watson(ols_model_cleaned.resid)
print(f'Durbin-Watson Test statistic (Cleaned): {dw_test_cleaned}')

vif_data_cleaned = pd.DataFrame()
vif_data_cleaned['feature'] = X_cleaned.columns
vif_data_cleaned['VIF'] = [variance_inflation_factor(X_cleaned_const.values, i) for i in range(X_cleaned_const.shape[1])]

print(vif_data_cleaned)
